In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import sys; sys.path.insert(0, '../')
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
import mne
from esinet import Simulation
from esinet.forward import get_info, create_forward_model
from esinet.util import unpack_fwd
pp = dict(surface='white', hemi='both')

# Get Forward Model

In [2]:
info = get_info(kind='biosemi64')
fwd = create_forward_model(info=info, sampling='ico3')

leadfield, pos = unpack_fwd(fwd)[1:3]
n_chans, n_dipoles = leadfield.shape

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    2.2s remaining:    3.8s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    2.3s remaining:    1.3s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    2.4s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished


# Get sample data

In [3]:
settings = dict(number_of_sources=3, extents=(25, 40), duration_of_trial=0.01, target_snr=25)

sim = Simulation(fwd, info, settings).simulate(2)
stc = sim.source_data[0]
evoked = sim.eeg_data[0].average()

brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title',
               font_size=14)

-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


  0%|          | 0/2 [00:00<?, ?it/s]c:\Users\lukas\virtualenvs\invertenv\lib\site-packages\esinet\simulation.py:387: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.simulation_info = self.simulation_info.append(d, ignore_index=True)
 50%|█████     | 1/2 [00:00<00:00,  1.33it/s]c:\Users\lukas\virtualenvs\invertenv\lib\site-packages\esinet\simulation.py:387: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.simulation_info = self.simulation_info.append(d, ignore_index=True)
100%|██████████| 2/2 [00:00<?, ?it/s]


source data shape:  (1284, 10) (1284, 10)


100%|██████████| 2/2 [00:00<00:00, 487.94it/s]


Using pyvistaqt 3d backend.

Using control points [1.25272357e-09 3.12407916e-09 5.92727705e-08]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# Regularization Optimizations

In [ ]:
from invert.evaluate import corr, nmse
from invert.solvers.lucas import SolverLUCAS

solver = SolverLUCAS()
solver.make_inverse_operator(fwd, evoked, verbose=0)
solver.optimize_weights(fwd, info)
solver.plot_weights()
stc_hat = solver.apply_inverse_operator(evoked)
stc_hat.plot(**pp, brain_kwargs=dict(title=solver.name))

print("Mean correlation: ", np.mean(corr(stc_hat.data, stc.data)))
print("Mean NMSE: ", np.mean(nmse(stc_hat.data, stc.data)))

# Temporal Context
# stc_hat_cbd = contextualize_bd(stc_hat, fwd, fast=True, lstm_look_back=10, verbose=1)
# stc_hat_cbd.plot(**pp, brain_kwargs=dict(title="c"+solver.name))

# print("Mean correlation: ", np.mean(corr(stc_hat_cbd.data, stc.data)))
# print("Mean NMSE: ", np.mean(nmse(stc_hat_cbd.data, stc.data)))


# # FOCUSS:
# stc_hat_focuss = focuss(stc_hat_cbd, evoked, fwd)
# stc_hat_focuss.plot(**pp, brain_kwargs=dict(title="c"+solver.name+" FOCUSS"))

# print("Mean correlation: ", np.mean(corr(stc_hat_focuss.data, stc.data)))
# print("Mean NMSE: ", np.mean(nmse(stc_hat_focuss.data, stc.data)))

In [5]:
from invert import Solver
from invert.evaluate import corr, nmse
from invert.adapters import contextualize_bd, focuss, s_focuss

solver_ = Solver(solver="laur")

solver_.make_inverse_operator(fwd, alpha="auto")
stc_hat = solver_.apply_inverse_operator(evoked)
# stc_hat.data[np.isnan(stc_hat.data)] = 0
stc_hat.plot(**pp, brain_kwargs=dict(title=solver_.name))
print("Mean correlation: ", np.mean(corr(stc_hat.data, stc.data)))
print("Mean NMSE: ", np.mean(nmse(stc_hat.data, stc.data)))

stc_hat_sf = s_focuss(stc_hat, evoked, fwd, verbose=1)
stc_hat_sf.plot(**pp, brain_kwargs=dict(title=solver_.name + " shrinking FOCUSS"))
print("Mean correlation: ", np.mean(corr(stc_hat_sf.data, stc.data)))
print("Mean NMSE: ", np.mean(nmse(stc_hat_sf.data, stc.data)))

stc_hat_sf = focuss(stc_hat, evoked, fwd, verbose=1)
stc_hat_sf.plot(**pp, brain_kwargs=dict(title=solver_.name + " FOCUSS"))
print("Mean correlation: ", np.mean(corr(stc_hat_sf.data, stc.data)))
print("Mean NMSE: ", np.mean(nmse(stc_hat_sf.data, stc.data)))

# # Temporal Context
# stc_hat_cbd = contextualize_bd(stc_hat, fwd, fast=True, lstm_look_back=10, verbose=1)
# stc_hat_cbd.plot(**pp, brain_kwargs=dict(title="c"+solver_.name))

# print("Mean correlation: ", np.mean(corr(stc_hat_cbd.data, stc.data)))
# print("Mean NMSE: ", np.mean(nmse(stc_hat_cbd.data, stc.data)))


# # FOCUSS:
# stc_hat_focuss = focuss(stc_hat_cbd, evoked, fwd)
# stc_hat_focuss.plot(**pp, brain_kwargs=dict(title="c"+solver_.name+" FOCUSS"))

# print("Mean correlation: ", np.mean(corr(stc_hat_focuss.data, stc.data)))
# print("Mean NMSE: ", np.mean(nmse(stc_hat_focuss.data, stc.data)))


Using control points [1.00295540e-09 1.16641942e-09 2.91248426e-09]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
Mean correlation:  0.14074433207742693
Mean NMSE:  0.023023068486939736
-- number of adjacent vertices : 1284
Shrinking FOCUSS:

Time step 0/10
Time step 1/10
Time step 2/10
Time step 3/10
Time step 4/10
Time step 5/10
Time step 6/10
Time step 7/10
Time step 8/10
Time step 9/10
Using control points [5.09529476e-59 1.25247081e-58 1.12653343e-55]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
Mean correlation:  -0.003055457928142368
Mean NMSE:  0.005667790421865522
FOCUSS:

Time step 0/10
Iteration 0
Iteration 1
Iteration 2
converged at repeti

# Adapt

In [4]:
from invert.adapters import contextualize_bd, contextualize

stc_hat_cbd = contextualize_bd(stc_hat, fwd, lstm_look_back=10, num_units=128, num_epochs=50, verbose=0)
stc_hat_cbd.plot(**pp, brain_kwargs=dict(title="c"+solver.name))

print("Mean correlation: ", np.mean(corr(stc_hat_cbd.data, stc.data)))
print("Mean NMSE: ", np.mean(nmse(stc_hat_cbd.data, stc.data)))

NameError: name 'stc_hat' is not defined

# Test/ Evaluation

In [35]:
from invert import Solver
from invert.config import all_solvers
from invert.evaluate import nmse, corr
settings = dict(number_of_sources=(1,10), extents=(1, 40), duration_of_trial=1, target_snr=(1,25))


# errors_corr = {sname: [] for sname in all_solvers}
# errors_nmse = {sname: [] for sname in all_solvers}

# solvers = dict()
for i in range(30):
    # print(i)
    sim = Simulation(fwd, info, settings).simulate(2)
    stc = sim.source_data[0]
    evoked = sim.eeg_data[0].average()

    for solver_name in all_solvers:
        print(solver_name)
        solver = Solver(solver=solver_name)
        if (not solver_name in solvers) or ("sparse" in solver_name.lower() or "bayes" in solver_name.lower()):
            solvers[solver_name] = solver.make_inverse_operator(fwd, evoked, alpha="auto")
        stc_hat = solvers[solver_name].apply_inverse_operator(evoked)
        # stc_hat.plot(**pp, brain_kwargs=dict(title=solver.name))
        error = np.mean(corr(stc.data, stc_hat.data))
        errors_corr[solver_name].append( error )
        
        error = np.mean(nmse(stc.data, stc_hat.data))
        errors_nmse[solver_name].append( error )
        
# import pickle as pkl
# fn = "errors.pkl"
# with open(fn, 'wb') as f:
#     pkl.dump([errors_corr, errors_nmse], f)

-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 250.01it/s]


source data shape:  (1284, 1000) (1284, 1000)


100%|██████████| 2/2 [00:00<00:00,  8.62it/s]


MNE
wMNE
dSPM
LORETA
sLORETA
eLORETA
LAURA
Backus-Gilbert
S-MAP
Multiple Sparse Priors
Bayesian LORETA
Bayesian MNE
Bayesian Beamformer
Bayesian Beamformer LORETA
Fully-Connected
1/1 [==============================] - 0s 156ms/step


...

source data shape:  (1284, 1000) (1284, 1000)


100%|██████████| 2/2 [00:00<00:00, 10.26it/s]


MNE
wMNE
dSPM
LORETA
sLORETA
eLORETA
LAURA
Backus-Gilbert
S-MAP
Multiple Sparse Priors
Bayesian LORETA
Bayesian MNE
Bayesian Beamformer
Bayesian Beamformer LORETA
Fully-Connected
1/1 [==============================] - 0s 21ms/step


c:\Users\lukas\virtualenvs\invertenv\lib\site-packages\scipy\optimize\_minimize.py:863: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "


LUCAS
Applying  Minimum Norm Estimate
Applying  Weighted Minimum Norm Estimate
Applying  Dynamic Statistical Parametric Mapping
Applying  Low Resolution Tomography
Applying  Standardized Low Resolution Tomography
Applying  Exact Low Resolution Tomography
Applying  Local Auto-Regressive Average
Applying  S-MAP
Applying  Fully-Connected
1/1 [==============================] - 0s 20ms/step


In [36]:
import pandas as pd
import seaborn as sns
sns.set(font_scale=0.8)
df = pd.DataFrame(errors)
sorted_index = df.median().sort_values().index
df = df[sorted_index]

plt.figure()
sns.boxplot(data=df)
plt.title("Correlation with ground truth")


df_mean_var = pd.concat([df.mean(), df.std()], axis=1)
df_mean_var = df_mean_var.rename(columns={0: "Median", 1: "Variance"})
df_mean_var["MedVar"] = df_mean_var["Median"] / df_mean_var["Variance"]
df_mean_var["Method"] = df_mean_var.index
display(df_mean_var)

plt.figure()
sns.scatterplot(x="Median", y="Variance", hue="Method", size="MedVar", data=df_mean_var)
plt.xlabel("Median")
plt.ylabel("Variance")


,Median,Variance,MedVar,Method
Bayesian Beamformer,0.033344,0.100641,0.331314,Bayesian Beamformer
Backus-Gilbert,0.020588,0.141354,0.145648,Backus-Gilbert
wMNE,0.078268,0.031768,2.463740,wMNE
Bayesian LORETA,0.116085,0.108286,1.072021,Bayesian LORETA
Multiple Sparse Priors,0.115361,0.085370,1.351295,Multiple Sparse Priors
dSPM,0.126188,0.050529,2.497370,dSPM
LUCAS,0.127139,0.050733,2.506054,LUCAS
LORETA,0.176581,0.157067,1.124241,LORETA
Bayesian MNE,0.140912,0.099831,1.411510,Bayesian MNE
S-MAP,0.182043,0.156817,1.160861,S-MAP


Text(0, 0.5, 'Variance')

In [ ]:
from invert.solvers.multiple_sparse_priors import SolverMultipleSparsePriors
from invert.solvers.loreta import SolverLORETA, SolverSLORETA, SolverELORETA
from invert.solvers.wrop import SolverBackusGilbert, SolverLAURA
from invert.solvers.smap import SolverSMAP
from invert.solvers.minimum_norm_estimates import SolverDynamicStatisticalParametricMapping, SolverWeightedMinimumNorm, SolverMinimumNorm
solvers = [SolverMultipleSparsePriors, SolverLORETA, SolverSLORETA, SolverELORETA, SolverBackusGilbert, SolverLAURA, SolverSMAP, SolverDynamicStatisticalParametricMapping, SolverWeightedMinimumNorm, SolverMinimumNorm]

for solver in solvers:
    solver_ = solver()
    if solver_.name == "Multiple Sparse Priors":
        solver_.make_inverse_operator(fwd, evoked, alpha='auto')
    else:
        solver_.make_inverse_operator(fwd, alpha='auto')
    stc_hat = solver_.apply_inverse_operator(evoked)
    stc_hat.plot(**pp, brain_kwargs=dict(title=solver_.name))



In [ ]:
from mne.minimum_norm import make_inverse_operator as mne_inverse
from mne.minimum_norm import apply_inverse as mne_apply
from mne import make_ad_hoc_cov
noise_cov = make_ad_hoc_cov(evoked.info, verbose=0)
mne_io = mne_inverse(evoked.info, fwd, noise_cov=noise_cov, fixed=True, loose=0, depth=0, verbose=0)
stc_hat = mne_apply(evoked, mne_io, method="MNE", verbose=0)
stc_hat.plot(**pp)